In [1]:
from datetime import datetime
import json
import pandas as pd
from prince import MCA
from random import randint
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import BayesianRidge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, RobustScaler, StandardScaler
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor

In [2]:
# Load the data with the modified data types

with open('../raw_data/dtypes.json', 'r') as file:
    dtypes = json.load(file)
df_train = pd.read_csv("../raw_data/exp_train.csv", dtype=dtypes)
df_test = pd.read_csv("../raw_data/exp_test.csv", dtype=dtypes)
#df_train.info()


In [3]:
# Store the random seed in a log file then generate the kfold splits

rs_log = open("rs_log.txt", 'a')
rs_log.write(f"\n{datetime.now()}, rs = {randint(0,1000)}")
#rs_log.write("\n"+str(datetime.now())+", rs = "+str(random.randint(0,1000)))
rs_log.close()
with open("rs_log.txt", 'r') as rs_log:
    rs = int(rs_log.readlines()[-1].split(" ")[-1])

n_splits = 5    
kfold = KFold(n_splits = n_splits, shuffle = True, random_state=rs)
kfold_splits = kfold.split(df_train)   

# Name the splits

dfs_tt, dfs_ho = [], []
for train_index, test_index in kfold_splits:
    dfs_tt.append(df_train.iloc[train_index])
    dfs_ho.append(df_train.iloc[test_index])     

In [4]:
# Impute `GarageYrBlt` with the year before the earliest
 
for i in range(n_splits):
    min_yr = dfs_tt[i]['GarageYrBlt'].min()
    yr_impute = SimpleImputer(strategy='constant', fill_value=min_yr-1)
    dfs_tt[i].loc[:,'GarageYrBlt'] = pd.DataFrame(yr_impute.fit_transform(dfs_tt[i][['GarageYrBlt']]))
    dfs_ho[i].loc[:,'GarageYrBlt'] = pd.DataFrame(yr_impute.transform(dfs_ho[i][['GarageYrBlt']]))

In [5]:
# Preprocessing pipelines for the numerical features 

num_med_cols = ['LotFrontage', 'LotArea', #'LotShape', 
                'OverallQual', 'OverallCond', 'YearBuilt', #'ExterQual', 
                #'ExterCond',
                   #'BsmtQual', 
                   'TotalBsmtSF', #'HeatingQC', 
                   '1stFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 
                   'HalfBath', 'BedroomAbvGr', #'KitchenQual', 
                   'TotRmsAbvGrd', 'Fireplaces', #'GarageFinish', 
                   'GarageCars', 'GarageArea', 'YrSold']
num_mode_cols = [#'LandSlope',  
    'YearRemodAdd', 'MasVnrArea', #'BsmtCond', 
    #'BsmtExposure', 
    # 'BsmtFinType1', 
                 'BsmtFinSF1', #'BsmtFinType2', 
                 'BsmtFinSF2', 'BsmtUnfSF', '2ndFlrSF', 'LowQualFinSF', 'BsmtHalfBath', 
                 'KitchenAbvGr', #'Functional', 
                 #'FireplaceQu', 
                 'GarageYrBlt', #'GarageQual', 
                 #'GarageCond', 
                 # 'PavedDrive', 
                 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', #'PoolQC', 
                 #'Fence', 
                 'MiscVal']
len_num_cols = len(num_med_cols) + len(num_mode_cols)

pipe_num_med = Pipeline([('impute', IterativeImputer(estimator=BayesianRidge(),#DecisionTreeRegressor(),
                                                     max_iter=50,
                                                     initial_strategy='median')),
                         #('outliers', RobustScaler()),
                         ('range', MinMaxScaler())])

pipe_num_mode = Pipeline([('impute', IterativeImputer(estimator=BayesianRidge(),#DecisionTreeRegressor(),
                                                     max_iter=50,
                                                     initial_strategy='most_frequent')),
                         #('outliers', RobustScaler()),
                         ('range', MinMaxScaler())])

In [6]:
cat_cols = df_train.select_dtypes(include = 'object').drop(columns=['Id']).columns.to_list()

# OHE first to get the max number of components

ohe = OneHotEncoder(handle_unknown='ignore')
new_cat_cols = []
for i in range(n_splits): 
    id_tt = dfs_tt[i]['Id']
    id_ho = dfs_ho[i]['Id']
    ohe_transformed_tt = pd.DataFrame(ohe.fit_transform(dfs_tt[i][cat_cols]).toarray())
    ohe_transformed_ho = pd.DataFrame(ohe.transform(dfs_ho[i][cat_cols]).toarray())
    prices_tt = dfs_tt[i]['SalePrice']
    prices_ho = dfs_ho[i]['SalePrice']
    cols1 = num_med_cols + num_mode_cols
    cols2 = ohe_transformed_tt.columns.to_list()
    cols3 = ['SalePrice']
    dfs_tt[i] = pd.concat([id_tt.reset_index(drop=True),
                           dfs_tt[i][cols1].reset_index(drop=True), 
                           ohe_transformed_tt.reset_index(drop=True), 
                           prices_tt.reset_index(drop=True)], 
                           axis=1, ignore_index=True)
    dfs_tt[i].columns = ['Id'] + cols1 + [str(x) for x in cols2] + cols3
    dfs_ho[i] = pd.concat([id_ho.reset_index(drop=True),
                           dfs_ho[i][cols1].reset_index(drop=True), 
                           ohe_transformed_ho.reset_index(drop=True), 
                           prices_ho.reset_index(drop=True)], 
                           axis=1, ignore_index=True)
    dfs_ho[i].columns = ['Id'] + cols1 + [str(x) for x in cols2] + cols3
    new_cat_cols.append([str(x) for x in cols2])
    # print(f"tt{i}: {dfs_tt[i]}")
    # print(f"ho{i}: {dfs_ho[i]}")    

# Make the cat cols pipeline

pipe_cat = MCA(n_components=len(new_cat_cols[i])-1, one_hot=False)

# # Put it all together

# ct = ColumnTransformer([('num_med', pipe_num_med, num_med_cols),
#                             ('num_mode', pipe_num_mode, num_mode_cols),
#                             ('cat', pipe_cat, new_cat_cols[i])],
#                             remainder='passthrough')

In [7]:
# Run the pipelines (tried ColumnTransformer and it was problematic)

# n_comps_list = []
for i in range(n_splits):
    print(f"tt{i} before: {dfs_tt[i]}")
    print(f"ho{i} before: {dfs_ho[i]}")
    tt_num_med_transf = pd.DataFrame(pipe_num_med.fit_transform(dfs_tt[i][num_med_cols]))
    ho_num_med_transf = pd.DataFrame(pipe_num_med.transform(dfs_ho[i][num_med_cols]))
    tt_num_mode_transf = pd.DataFrame(pipe_num_mode.fit_transform(dfs_tt[i][num_mode_cols]))
    ho_num_mode_transf = pd.DataFrame(pipe_num_mode.transform(dfs_ho[i][num_mode_cols]))
    tt_cat_tranf = pd.DataFrame(pipe_cat.fit_transform(dfs_tt[i][new_cat_cols[i]]))
    ho_cat_tranf = pd.DataFrame(pipe_cat.transform(dfs_ho[i][new_cat_cols[i]]))
    dfs_tt[i] = pd.concat([dfs_tt[i]['Id'], 
                           tt_num_med_transf,
                           tt_num_mode_transf,
                           tt_cat_tranf,
                           dfs_tt[i]['SalePrice']], axis=1)
    dfs_ho[i] = pd.concat([dfs_ho[i]['Id'], 
                           ho_num_med_transf,
                           ho_num_mode_transf,
                           ho_cat_tranf,
                           dfs_ho[i]['SalePrice']], axis=1)
    print(f"tt{i} after: {dfs_tt[i]}")
    print(f"ho{i} after: {dfs_ho[i]}")
    
#     # Determine the number of components to keep from MCA 

#     exp_var = 0.9
#     eigenvalues = pipe_cat.eigenvalues_
#     total_inertia = eigenvalues.sum()
#     evr_list = eigenvalues / total_inertia
#     exp_var_sum = 0
#     n_comp = 0
#     for j in range(len(evr_list)):
#         exp_var_sum = exp_var_sum + evr_list[j]
#         if exp_var_sum <= exp_var:
#             n_comp += 1
#         else:
#             break 
#     n_comps_list.append(n_comp) 
    
# # Cut the extraneous features in the categorical dataframe

# n_cols = 1 + len_num_cols + max(n_comps_list)
# print(f"n_cols = {n_cols}")
# print(f"shape of tt is {dfs_tt[0].shape}")
# for i in range(n_splits):
#     dfs_tt[i] = dfs_tt[i].iloc[:, list(range(n_cols)) + [-1]]
#     dfs_tt[i].columns = ['Id'] + num_med_cols + num_mode_cols + list(range(max(n_comps_list))) + ['SalePrice']
#     dfs_ho[i] = dfs_ho[i].iloc[:, list(range(n_cols)) + [-1]]  
#     dfs_ho[i].columns = ['Id'] + num_med_cols + num_mode_cols + list(range(max(n_comps_list))) + ['SalePrice']

# print(f"tt0 finally: {dfs_tt[0]}")
# print(f"ho0 finally: {dfs_ho[0]}")

tt0 before:         Id  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0        1         65.0     8450            7            5       2003   
1        2         80.0     9600            6            8       1976   
2        3         68.0    11250            7            5       2001   
3        4         60.0     9550            7            5       1915   
4        5         84.0    14260            8            5       2000   
...    ...          ...      ...          ...          ...        ...   
1163  1456         62.0     7917            6            5       1999   
1164  1457         85.0    13175            6            6       1978   
1165  1458         66.0     9042            7            9       1941   
1166  1459         68.0     9717            5            6       1950   
1167  1460         75.0     9937            5            6       1965   

      TotalBsmtSF  1stFlrSF  GrLivArea  BsmtFullBath  ...  277  278  279  280  \
0           856.0       856   

c:\Users\leyjf\Dropbox\github\HousingPrices\housing_prices_venv\Lib\site-packages\sklearn\impute\_base.py:598: UserWarning: Skipping features without any observed values: [9]. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
c:\Users\leyjf\Dropbox\github\HousingPrices\housing_prices_venv\Lib\site-packages\sklearn\impute\_base.py:598: UserWarning: Skipping features without any observed values: [9]. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


tt0 after:         Id         0         1         2      3         4         5         6  \
0        1  0.124803  0.043768  0.666667  0.500  0.949275  0.140098  0.119780   
1        2  0.167349  0.050808  0.555556  0.875  0.753623  0.206547  0.212942   
2        3  0.133312  0.060908  0.666667  0.500  0.934783  0.150573  0.134465   
3        4  0.110621  0.050502  0.666667  0.500  0.311594  0.123732  0.143873   
4        5  0.178695  0.079334  0.777778  0.500  0.927536  0.187398  0.186095   
...    ...       ...       ...       ...    ...       ...       ...       ...   
1163  1456  0.116293  0.040506  0.555556  0.500  0.920290  0.155974  0.142038   
1164  1457  0.181531  0.072692  0.555556  0.625  0.768116  0.252373  0.399036   
1165  1458  0.127639  0.047392  0.666667  1.000  0.500000  0.188543  0.195961   
1166  1459  0.133312  0.051524  0.444444  0.625  0.565217  0.176432  0.170721   
1167  1460  0.153167  0.052871  0.444444  0.625  0.673913  0.205565  0.211565   

             7  

c:\Users\leyjf\Dropbox\github\HousingPrices\housing_prices_venv\Lib\site-packages\sklearn\impute\_base.py:598: UserWarning: Skipping features without any observed values: [9]. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
c:\Users\leyjf\Dropbox\github\HousingPrices\housing_prices_venv\Lib\site-packages\sklearn\impute\_base.py:598: UserWarning: Skipping features without any observed values: [9]. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


tt1 after:         Id         0         1         2      3         4         5         6  \
0        1  0.150685  0.033420  0.666667  0.500  0.949275  0.140098  0.119780   
1        2  0.202055  0.038795  0.555556  0.875  0.753623  0.206547  0.212942   
2        3  0.160959  0.046507  0.666667  0.500  0.934783  0.150573  0.134465   
3        4  0.133562  0.038561  0.666667  0.500  0.311594  0.123732  0.143873   
4        5  0.215753  0.060576  0.777778  0.500  0.927536  0.187398  0.186095   
...    ...       ...       ...       ...    ...       ...       ...       ...   
1163  1456  0.140411  0.030929  0.555556  0.500  0.920290  0.155974  0.142038   
1164  1457  0.219178  0.055505  0.555556  0.625  0.768116  0.252373  0.399036   
1165  1458  0.154110  0.036187  0.666667  1.000  0.500000  0.188543  0.195961   
1166  1459  0.160959  0.039342  0.444444  0.625  0.565217  0.176432  0.170721   
1167  1460  0.184932  0.040370  0.444444  0.625  0.673913  0.205565  0.211565   

             7  

...

tt2 after:         Id         0         1         2      3         4         5         6  \
0        1  0.150685  0.033420  0.666667  0.500  0.949275  0.140098  0.119780   
1        3  0.160959  0.046507  0.666667  0.500  0.934783  0.150573  0.134465   
2        6  0.219178  0.059899  0.444444  0.500  0.876812  0.130278  0.106012   
3        7  0.184932  0.041057  0.777778  0.500  0.956522  0.275941  0.312070   
4        8  0.174223  0.042450  0.666667  0.625  0.731884  0.181178  0.177375   
...    ...       ...       ...       ...    ...       ...       ...       ...   
1163  1456  0.140411  0.030929  0.555556  0.500  0.920290  0.155974  0.142038   
1164  1457  0.219178  0.055505  0.555556  0.625  0.768116  0.252373  0.399036   
1165  1458  0.154110  0.036187  0.666667  1.000  0.500000  0.188543  0.195961   
1166  1459  0.160959  0.039342  0.444444  0.625  0.565217  0.176432  0.170721   
1167  1460  0.184932  0.040370  0.444444  0.625  0.673913  0.205565  0.211565   

             7  

c:\Users\leyjf\Dropbox\github\HousingPrices\housing_prices_venv\Lib\site-packages\sklearn\impute\_base.py:598: UserWarning: Skipping features without any observed values: [9]. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
c:\Users\leyjf\Dropbox\github\HousingPrices\housing_prices_venv\Lib\site-packages\sklearn\impute\_base.py:598: UserWarning: Skipping features without any observed values: [9]. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


tt3 after:         Id         0         1         2         3         4         5  \
0        1  0.150685  0.033420  0.666667  0.428571  0.955224  0.140098   
1        2  0.202055  0.038795  0.555556  0.857143  0.753731  0.206547   
2        4  0.133562  0.038561  0.666667  0.428571  0.298507  0.123732   
3        5  0.215753  0.060576  0.777778  0.428571  0.932836  0.187398   
4        6  0.219178  0.059899  0.444444  0.428571  0.880597  0.130278   
...    ...       ...       ...       ...       ...       ...       ...   
1163  1453  0.047945  0.011101  0.444444  0.428571  0.970149  0.089525   
1164  1454  0.236301  0.074398  0.444444  0.428571  0.977612  0.186579   
1165  1455  0.140411  0.028979  0.666667  0.428571  0.962687  0.199836   
1166  1456  0.140411  0.030929  0.555556  0.428571  0.925373  0.155974   
1167  1459  0.160959  0.039342  0.444444  0.571429  0.559701  0.176432   

             6         7    8  ...       278       279       280       281  \
0     0.119780  0.2592

c:\Users\leyjf\Dropbox\github\HousingPrices\housing_prices_venv\Lib\site-packages\sklearn\impute\_base.py:598: UserWarning: Skipping features without any observed values: [9]. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
c:\Users\leyjf\Dropbox\github\HousingPrices\housing_prices_venv\Lib\site-packages\sklearn\impute\_base.py:598: UserWarning: Skipping features without any observed values: [9]. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


tt4 after:         Id         0         1         2      3         4         5         6  \
0        2  0.202055  0.037999  0.555556  0.875  0.753623  0.393637  0.311625   
1        3  0.160959  0.045718  0.666667  0.500  0.934783  0.286962  0.191877   
2        4  0.133562  0.037765  0.666667  0.500  0.311594  0.235808  0.206232   
3        5  0.215753  0.059798  0.777778  0.500  0.927536  0.357143  0.270658   
4        6  0.219178  0.059120  0.444444  0.500  0.876812  0.248284  0.148459   
...    ...       ...       ...       ...    ...       ...       ...       ...   
1163  1452  0.195205  0.036418  0.777778  0.500  0.985507  0.490643  0.422269   
1164  1454  0.236301  0.073631  0.444444  0.500  0.971014  0.355583  0.268908   
1165  1457  0.219178  0.054723  0.555556  0.625  0.768116  0.480973  0.595588   
1166  1458  0.154110  0.035389  0.666667  1.000  0.500000  0.359326  0.285714   
1167  1460  0.184932  0.039576  0.444444  0.625  0.673913  0.391765  0.309524   

             7  

In [8]:
# Save the preprocessed data

print(dfs_tt[0].shape)
for i in range(n_splits):
    dfs_tt[i].to_csv(f"preproc_tt_fold_{i}.csv", index=False)
    dfs_ho[i].to_csv(f"preproc_ho_fold_{i}.csv", index=False)

(1168, 320)
